<a href="https://colab.research.google.com/github/julieannaqi/julieannaqi/blob/main/dbscan_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install memory_profiler

In [ ]:
import pandas as pd

# Replace 'juile.csv' with the actual filename if it's different
df = pd.read_csv('/content/sampledata.csv')

# Display the contents of the CSV file
print(df)
df_slice = df.iloc[:1000, :10]


                          ioc_match_id     event_time       src_host  \
0      1773_362000776412_0_60382090314  1701618689736  10.15.149.168   
1      1773_361424542980_0_59299513186  1701589169373  10.15.149.168   
2      1773_361424542980_0_59327455359  1701589169373  10.15.149.168   
3      1773_361424542980_0_59327639236  1701589169373  10.15.149.168   
4      1773_362000776412_0_60440942415  1701618689736  10.15.149.168   
...                                ...            ...            ...   
99995  1773_361569283028_0_60382090483  1701596405801  10.20.100.250   
99996  1773_361569283028_0_60724428162  1701596405801  10.20.100.250   
99997  1773_361569283121_0_60362960162  1701596405801   10.20.106.39   
99998  1773_361569283121_0_60374602795  1701596405801   10.20.106.39   
99999  1773_361569283121_0_60650052021  1701596405801   10.20.106.39   

            indicator  confidence severity indicator_type  originator  \
0        77.90.185.71          87      low             ip     

In [ ]:
print(df.shape, df_slice.shape)

(100000, 23) (1000, 10)


In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
import h5py
import pyarrow as pa
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.metaestimators import if_delegate_has_method

In [ ]:
def calculate_metrics(data, predicted_labels):
        # homogeneity = homogeneity_score(true_labels, predicted_labels)
        # completeness = completeness_score(true_labels, predicted_labels)
        # v_measure = v_measure_score(true_labels, predicted_labels)
        # adj_rand_index = adjusted_rand_score(true_labels, predicted_labels)
        # adj_mutual_info = adjusted_mutual_info_score(true_labels, predicted_labels)
        # we don't know the true label at the moment, so we can only return the sillhouette score.
        silhouette_coeff = silhouette_score(data, predicted_labels)

        # return {
        #     "Homogeneity": homogeneity,
        #     "Completeness": completeness,
        #     "V-Measure": v_measure,
        #     "Adjusted Rand Index": adj_rand_index,
        #     "Adjusted Mutual Information": adj_mutual_info,
        #     "Silhouette Coefficient": silhouette_coeff
        # }
        return silhouette_coeff


In [ ]:
df = df_slice

In [ ]:
non_numeric_cols = df.select_dtypes(exclude=['number']).columns.tolist()

# Numeric columns for imputation and scaling
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

# ColumnTransformer to handle different transformations for different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), non_numeric_cols),
        ('imputer', SimpleImputer(strategy='constant', fill_value=-1), numeric_cols),  # Apply imputer to numeric columns
        ('scaler', StandardScaler(), numeric_cols)  # Apply scaler to numeric columns
    ]
)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Fit and transform your data using the pipeline
processed_data = pipeline.fit_transform(df)

In [ ]:
from scipy.sparse import csr_matrix

# Assuming 'df' is your Pandas DataFrame containing sparse data

# Convert the DataFrame to a sparse matrix
sparse_matrix = csr_matrix(processed_data)

In [ ]:
%%time
%load_ext memory_profiler
%memit
from sklearn.cluster import DBSCAN
import pandas as pd

# Assuming you have already defined and fitted your pipeline to obtain processed_data

# Create DBSCAN object with your desired parameters
dbscan = DBSCAN(eps=0.1, min_samples=10)  # You might need to adjust the epsilon (eps) and min_samples

# Fit DBSCAN to your processed data
labels = dbscan.fit_predict(sparse_matrix)

peak memory: 261.85 MiB, increment: 0.88 MiB
CPU times: user 525 ms, sys: 91.7 ms, total: 617 ms
Wall time: 502 ms


In [ ]:
print(f'the silhouette score for dbscan alone was {calculate_metrics(processed_data, labels)}')

the silhouette score for dbscan alone was 0.9999627374626776


In [ ]:
from scipy.spatial import cKDTree

In [ ]:
pip install Cython


In [ ]:
import numpy as np
from scipy.spatial import cKDTree

def dbscan_kdtree_optimized(data, eps, min_pts):
    n = data.shape[0]
    visited = np.zeros(n, dtype=bool)
    labels = np.zeros(n, dtype=int)
    cluster_id = 0
    tree = cKDTree(data, compact_nodes=True)

    def expand_cluster(point_idx, neighbors, cluster_id):
        labels[point_idx] = cluster_id
        for point in neighbors:
            if not visited[point]:
                visited[point] = True
                new_neighbors = tree.query_ball_point(data[point], eps)
                if len(new_neighbors) >= min_pts:
                    neighbors.extend([n for n in new_neighbors if n not in neighbors])
            if labels[point] == -1:
                labels[point] = cluster_id
            elif labels[point] == 0:
                labels[point] = cluster_id
                new_point_neighbors = tree.query_ball_point(data[point], eps)
                if len(new_point_neighbors) >= min_pts:
                    neighbors.extend([n for n in new_point_neighbors if n not in neighbors])

    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        neighbors = tree.query_ball_point(data[i], eps)
        if len(neighbors) < min_pts:
            labels[i] = -1
        else:
            cluster_id += 1
            expand_cluster(i, neighbors, cluster_id)

    return labels


Let's try to optimize DBSCAN using kd trees, so we don't have to compute the euclidean distance between each point pair combination.

In [ ]:
data = processed_data.toarray()

In [ ]:
%%time
%memit
cluster_labels = dbscan_kdtree(data, eps=0.6, min_pts=10)
print(f'the silhouette score for dbscan using kdtree optimization was {calculate_metrics(data, cluster_labels)}')

peak memory: 283.47 MiB, increment: 0.00 MiB
the silhouette score for dbscan using kdtree optimization was -0.19665295913277306
CPU times: user 982 ms, sys: 61 ms, total: 1.04 s
Wall time: 1.34 s
